In [2]:
import pandas as pd
from model_tools import Preprocess
import os
from tqdm.notebook import tqdm

In [3]:
# convert 18K to 10K axis
#axis_10K = pd.read_csv('P:\\DATASETS\\10K\\final_datasets\\10K_spectra_EARTH_9mJ.csv', usecols=[0])

folder = 'P:\\DATASETS\\10K\\datasets for sensitivity calcs\\just preprocessed'
outfolder = 'P:\\DATASETS\\10K\\datasets for sensitivity calcs\\10K axis_AirPLS_norm3'
file_list = [f for f in os.listdir(folder) if 'spectra' in f]
for file in tqdm(file_list):
    spectra = pd.read_csv(f'{folder}\\{file}')
    
    # BLR
    spectra_blr = Preprocess.AirPLS(spectra)

    # norm5
    spectra_norm = Preprocess.norm3_SL_10K(spectra_blr)
    spectra_norm.to_csv(f'{outfolder}\\{file[:-4]}_formatted.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
inst_list = ['10K']#,'18K']

for inst in inst_list:
    folder = f'P:\\DATASETS\\{inst}\\final_datasets'
    outfolder = f'{folder}\\10K axis_AirPLS_norm3'

    spec_files = [f for f in os.listdir(folder) if (('spectra' in f) and ('no_outliers' not in f))]

    for file in tqdm(spec_files):
        df = pd.read_csv(f'{folder}\\{file}')

        # not going to resample to min step first
        # BLR
        df_blr = Preprocess.AirPLS(df)

        # norm5
        df_norm = Preprocess.norm3_SL_10K(df_blr)

        df_norm.to_csv(f'{outfolder}\\{file[:-4]}_formatted.csv', index=False)

  0%|          | 0/16 [00:00<?, ?it/s]